In [9]:
DATASET_DIR = ''
#GLOVE_DIR = './glove.6B/'
SAVE_DIR = './'

import os
import pandas as pd
import keras
from keras.callbacks import EarlyStopping
from keras.utils.np_utils import to_categorical

environment='L'


In [10]:

if environment == 'S':
    os.environ["CUDA_VISIBLE_DEVICES"] = '1'  
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
    import tensorflow as tf
    config = tf.ConfigProto()
    config.gpu_options.allow_growth =\
        True
    sess = tf.Session(config=config)
    keras.backend.set_session(sess)
elif environment == 'L':
    os.chdir("/Users/omerorhan/Desktop/lexinsight/AutomaticTextUnderstanding/venv/Source")

In [11]:


X = pd.read_csv(os.path.join(DATASET_DIR, 'essay_set1_merged.csv'), encoding='ISO-8859-1')
#X = (X[(X["essay_set"] == 1) | (X["essay_set"] == 2)])
X = (X[(X["essay_set"] == 1)])
print(X.columns)
#y = X[[ 'domain1_score', 'Content',
#       'Organization', 'Word Choice', 'Sentence Fluency', 'Conventions']]
y1=X[['rater1_domain1']]
y2=X[['Content']]
X = X.dropna(axis=1)
os.chdir("/Users/omerorhan/Desktop/lexinsight/AutomaticTextUnderstanding/venv/Source")


Index(['essay_id', 'essay_set', 'essay', 'rater1_domain1', 'Content',
       'Organization', 'Word Choice', 'Sentence Fluency', 'Conventions'],
      dtype='object')


Preprocessing the Data
We will preprocess all essays and convert them to feature vectors so that they can be fed into the RNN.

These are all helper functions used to clean the essays.

In [4]:
def to_one_hot(labels, dimension=6):
    
    results = np.zeros((len(labels), dimension))
    count=0;
    for i, label in enumerate(labels):
        #results[i, label] = 1.
        count=count+1
    print("sds"+str(count))
    return results
#y_train = to_one_hot(y_train)

#print(y)
#y=y-1
#y_train = to_categorical(y)
#print(y_train[0])

In [12]:
minimum_scores = [-1, 2, 1, 0, 0, 0, 0, 0, 0]
maximum_scores = [-1, 12, 6, 3, 3, 4, 4, 30, 60]

import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec

def essay_to_wordlist(essay_v, remove_stopwords):
    """Remove the tagged labels and word tokenize the sentence."""
    essay_v = re.sub("[^a-zA-Z]", " ", essay_v)
    words = essay_v.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return (words)

def essay_to_sentences(essay_v, remove_stopwords):
    """Sentence tokenize the essay and call essay_to_wordlist() for word tokenization."""
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(essay_v.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(essay_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

def makeFeatureVec(words, model, num_features):
    """Make Feature Vector from the words list of an Essay."""
    featureVec = np.zeros((num_features,),dtype="float32")
    num_words = 0.
    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set:
            num_words += 1
            featureVec = np.add(featureVec,model[word])        
    featureVec = np.divide(featureVec,num_words)
    return featureVec

def getAvgFeatureVecs(essays, model, num_features):
    """Main function to generate the word vectors for word2vec model."""
    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float32")
    for essay in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay, model, num_features)
        counter = counter + 1
    return essayFeatureVecs


Defining the model
Here we define a 2-Layer LSTM Model.

Note that instead of using sigmoid activation in the output layer we will use Relu since we are not normalising training labels.



In [13]:

from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K
from keras.utils.np_utils import to_categorical
from keras.layers import Input
from keras.models import Model

def get_model():
    """Define the model.""" 
    inputs = Input(shape=(1, 300))
    extract = LSTM(300, return_sequences=True)(inputs)
    class11 = LSTM(64)(extract)
    output1 = Dense(7, activation='sigmoid')(class11)
    output2 = Dense(7, activation='sigmoid')(class11)
    model = Model(inputs=inputs,outputs=[output1,output2])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    '''
    model = Sequential()
    model.add(LSTM(300, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4, return_sequences=False))
    model.add(Dropout(0.5))
    #model.add(Dense(6, activation='softmax'))
    model.add(Dense(6, activation='sigmoid'))
    #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    '''
    return model
'''
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K

def get_model():
    """Define the model."""
    model = Sequential()
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))

    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()

    return model
'''

'\nfrom keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten\nfrom keras.models import Sequential, load_model, model_from_config\nimport keras.backend as K\n\ndef get_model():\n    """Define the model."""\n    model = Sequential()\n    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))\n    model.add(LSTM(64, recurrent_dropout=0.4))\n    model.add(Dropout(0.5))\n    model.add(Dense(1, activation=\'relu\'))\n\n    model.compile(loss=\'mean_squared_error\', optimizer=\'rmsprop\', metrics=[\'mae\'])\n    model.summary()\n\n    return model\n'

Training Phase
Now we train the model on the dataset.

We will use 5-Fold Cross Validation and measure the Quadratic Weighted Kappa for each fold. We will then calculate Average Kappa for all the folds.

In [15]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import cohen_kappa_score

cv = KFold(n_splits=5,shuffle=True)
results = []
y_pred_list = []

count = 1
print(X.shape)
#for traincv, testcv in cv.split(X):
#    print("\n--------Fold {}--------\n".format(count))
#X_test, X_train, y_test, y_train = X.iloc[testcv], X.iloc[traincv], y.iloc[testcv], y.iloc[traincv]
#print(X_train.shape)
train_essays = X['essay']
#test_essays = X_test['essay']

sentences = []

for essay in train_essays:
        # Obtaining all sentences from the training essays.
        sentences += essay_to_sentences(essay, remove_stopwords = True)
        
# Initializing variables for word2vec model.
num_features = 300 
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3

print("Training Word2Vec Model...")
model = Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count,
                 window = context, sample = downsampling)

model.init_sims(replace=True)
model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

clean_train_essays = []

# Generate training and testing data word vectors.
for essay_v in train_essays:
    clean_train_essays.append(essay_to_wordlist(essay_v, remove_stopwords=True))
trainDataVecs = getAvgFeatureVecs(clean_train_essays, model, num_features)

#clean_test_essays = []
#for essay_v in test_essays:
#    clean_test_essays.append(essay_to_wordlist( essay_v, remove_stopwords=True ))
#testDataVecs = getAvgFeatureVecs( clean_test_essays, model, num_features )


trainDataVecs = np.array(trainDataVecs)
##testDataVecs = np.array(testDataVecs)
# Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
print("trainDataVecs"+str(trainDataVecs.shape))

trainDataVecs = np.reshape(trainDataVecs, (trainDataVecs.shape[0], 1, trainDataVecs.shape[1]))
#testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))
validsiz=200;
#y_train = y_train -1
#y_test = y_test-1
#x_val= testDataVecs[0:validsiz]
#y_val = y_test[0:validsiz]

#y_val = to_categorical(y_val, num_classes=6)    


#from sklearn.preprocessing import MultiLabelBinarizer
#one_hot = MultiLabelBinarizer()
#y_train = one_hot.fit_transform(y_train)
#y_train = to_one_hot(y_train)
#y_train = to_categorical(y_train, num_classes=6)

y_train = to_categorical(y1)
y_train2 = to_categorical(y2)

print(y_train)
#print("ytrain"+str(type(y_train)))
#print("trainDataVecs"+str(trainDataVecs.shape))

lstm_model = get_model()
#,validation_data=(x_val, y_val)
history = lstm_model.fit(trainDataVecs, y=[y_train,y_train2], batch_size=64, epochs=50)
#y_pred = lstm_model.predict(testDataVecs)
# Save any one of the 8 models.
if count == 1:
     lstm_model.save('final_lstm.h5')

# Round y_pred to the nearest integer.
#y_pred = np.around(y_pred)
'''
# Evaluate the model on the evaluation metric. "Quadratic mean averaged Kappa"
result = cohen_kappa_score(y_test.values,y_pred,weights='quadratic')
print("Kappa Score: {}".format(result))
results.append(result)
'''

import matplotlib.pyplot as plt
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()


plt.subplot(222)
valacc = history.history['val_acc']
acc = history.history['acc']
plt.plot(epochs, valacc, 'r', label='val_categorical_accuracy')
plt.plot(epochs, acc, 'y', label='categorical_accuracy')
plt.title('Val Cat. and Cat. Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

#break
count += 1


(1783, 9)


Training Word2Vec Model...


/anaconda3/envs/tf-cpu/lib/python3.5/site-packages/ipykernel_launcher.py:37: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


trainDataVecs(1783, 300)
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]
Instructions for updating:
Colocations handled automatically by placer.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1, 300)       0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 1, 300)       721200      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   (None, 64)           93440       lstm_1[0][0]                     
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 7)            455         lstm_2[0][0]                     
__________________________________________________________________________________________________
dense_2 (D

...

1472/1783 [=======================>......] - ETA: 0s - loss: 0.5487 - dense_1_loss: 0.2576 - dense_2_loss: 0.2911 - dense_1_acc: 0.8746 - dense_2_acc: 0.8602

1600/1783 [=========================>....] - ETA: 0s - loss: 0.5450 - dense_1_loss: 0.2552 - dense_2_loss: 0.2898 - dense_1_acc: 0.8759 - dense_2_acc: 0.8600

1728/1783 [============================>.] - ETA: 0s - loss: 0.5416 - dense_1_loss: 0.2541 - dense_2_loss: 0.2875 - dense_1_acc: 0.8766 - dense_2_acc: 0.8604

1783/1783 [==============================] - 1s 479us/step - loss: 0.5402 - dense_1_loss: 0.2532 - dense_2_loss: 0.2870 - dense_1_acc: 0.8774 - dense_2_acc: 0.8602


KeyError: 'val_loss'

In [43]:
model = keras.models.load_model("final_lstm.h5")  # Load Keras model we learned using KerasPractice 
test = trainDataVecs[1:2]
test_p = y_train[1:2]
print(test_p)
model.predict(test)

[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]


array([[1.8432736e-04, 6.3183904e-04, 5.1012635e-04, 2.4204850e-03,
        1.3749599e-03, 2.3709595e-02, 3.4940720e-02, 3.2466865e-01,
        2.4821348e-01, 2.5709864e-01, 8.0462538e-02, 2.5904387e-02]],
      dtype=float32)

In [45]:
X = pd.read_csv(os.path.join(DATASET_DIR, 'training_set_rel3.csv'), encoding='ISO-8859-1')
Xplus = pd.read_csv(os.path.join(DATASET_DIR, 'Prompt-1.csv'), encoding='ISO-8859-1')
X = (X[(X["essay_set"] == 1)])
#X = (X[(X["essay_set"] == 1)])
X = X.dropna(axis=1)
#newset = X.set_index('essay_id').join(Xplus.set_index('EssayID'))
newset =X.merge(Xplus,left_on='essay_id', right_on='EssayID')
print(newset.columns)
newset = newset[["essay_id","essay_set","essay", "rater1_domain1",'Content',
       'Organization', 'Word Choice', 'Sentence Fluency', 'Conventions']]
newset.to_csv("essay_set1_merged.csv",index=False);

Index(['essay_id', 'essay_set', 'essay', 'rater1_domain1', 'rater2_domain1',
       'domain1_score', 'domain1_score_copy', 'EssayID', 'Content',
       'Organization', 'Word Choice', 'Sentence Fluency', 'Conventions'],
      dtype='object')
